# Start Isaac Sim
Override the options as needed
DEFAULT_LAUNCHER_CONFIG= {'active_gpu': None, 'anti_aliasing': 3, 'denoiser': True, 'display_options': 3094, 'fast_shutdown': True, 'headless': True, 'height': 720, 'hide_ui': None, 'livesync_usd': None, 'max_bounces': 4, 'max_gpu_count': None, 'max_specular_transmission_bounces': 6, 'max_volume_bounces': 4, 'multi_gpu': True, 'open_usd': None, 'physics_gpu': 0, 'profiler_backend': [], 'renderer': 'RayTracedLighting', 'samples_per_pixel_per_frame': 64, 'subdiv_refinement_level': 0, 'sync_loads': True, 'width': 1280, 'window_height': 900, 'window_width': 1440}

[API](https://docs.omniverse.nvidia.com/py/isaacsim/source/extensions/omni.isaac.kit/docs/index.html#simulation-application-omni-isaac-kit)

Some ROS library need to import before starting Isaac.
It probably is some bug that will be addressed in future releases..

In [1]:
import sys
import signal

from isaacsim import SimulationApp

CONFIG = {
    'headless': False,
    'display_options': 3286
}

# Example for creating a RTX lidar sensor and publishing PointCloud2 data
simulation_app = SimulationApp(CONFIG)
simulation_app.update()

spinning = True
def sigint_handler(sig, frame):
    global spinning
    spinning = False
signal.signal(signal.SIGINT, sigint_handler)

kernel = get_ipython().kernel
kernel.pre_handler_hook = lambda: None
kernel.post_handler_hook = lambda: None

def spin(update_function, *args, **kwargs):
    global spinning
    print('gui can be used now. interrupt kernel to go back to notebook')
    while spinning:
        update_function(*args, **kwargs)
    spinning = True

def resume_gui():
    spin(lambda: simulation_app.update())

Starting kit application with the following args:  ['/f/ov/pkg/isaac-sim-4.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/f/ov/pkg/isaac-sim-4.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/f/ov/pkg/isaac-sim-4.1.0/kit', '--/persistent/app/viewport/displayOptions=3286', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/exts', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/apps', '--/physics/cudaDevice=0', '--portable']
Passing the following args to the base kit application:  ['-f', '/home/linh/.local/share/jupyter/runtime/kernel-5c4b082b-4489-41e8-be19-e3a1a7e3f59b.json']
[Warning] [omni.isaac.kit.simulation_app] Non-headless mode not supported with jupyt

Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.5
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4060" (8 GiB, sm_89, mempool enabled)
   Kernel cache:
     /home/linh/.cache/warp/1.2.1
 [ext: omni.kit.widget.text_editor-1.0.2] startup
[2.993s] [ext: omni.kit.viewport.window-106.0.8] startup
[3.015s] [ext: omni.kit.window.property-1.11.1] startup
[3.017s] [ext: omni.usdphysics-106.0.20] startup
[3.022s] [ext: omni.graph.tools-1.78.0] startup
[3.045s] [ext: omni.kit.viewport.utility-1.0.17] startup
[3.046s] [ext: omni.kit.property.usd-3.21.28] startup
[3.056s] [ext: omni.graph-1.135.0] startup
[3.181s] [ext: omni.graph.image.core-0.3.2] startup
[3.187s] [ext: omni.graph.ui-1.70.0] startup
[3.204s] [ext: omni.graph.action_core-1.1.4] startup
[3.215s] [ext: omni.kit.window.cursor-1.1.2] startup
[3.217s] [ext: omni.graph.image.nodes-1.0.2] startup
[3.219s] [ext: omni.graph.action_nodes-1.23.0] startup
[3.230s] [ext: omni.kit.viewport.menubar.core-106.0

In [4]:
import os
import glob
import json

import carb
import omni
from pxr import Gf, Sdf
import omni.graph.core as og
import omni.replicator.core as rep
from omni.isaac.core import SimulationContext, World
from omni.isaac.core.objects import *
from omni.isaac.core.utils import stage
from omni.isaac.core.utils.prims import *
from omni.isaac.core.utils.viewports import *
from omni.isaac.core.utils.rotations import euler_angles_to_quat
from omni.isaac.core.utils.stage import *
from omni.isaac.sensor import LidarRtx, IMUSensor
from omni.isaac.nucleus import get_assets_root_path
from omni.isaac.core.utils.types import *
import numpy as np


In [3]:

create_new_stage()
world = World(1.0/60, 1.0/60)
cube = DynamicCuboid('/cube', 'cube0')
world.scene.add(cube)
cube.prim.GetAttribute('physics:kinematicEnabled').Set(True)
world.reset()


[10.983s] Simulation App Startup Complete


2024-09-09 01:43:06 [13,705ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setLinearVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 232
2024-09-09 01:43:06 [13,706ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setAngularVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 256


In [6]:
DynamicState.__str__ = lambda self: f"DynamicState(pos: {self.position}, orient: {self.orientation}, lin_vel: {self.linear_velocity}, ang_vel: {self.angular_velocity})"

In [8]:
world.reset(soft=True)

# set_local_pose can update position and orientation of the kinematic object
# but it does not change the velocity of the object
for _ in range(10):
    p = np.random.rand(7)
    cube.set_local_pose(p[:3], p[3:])
    world.step()
    print(cube.get_current_dynamic_state())


2024-09-09 01:48:02 [309,789ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setLinearVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 232
2024-09-09 01:48:02 [309,789ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setAngularVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 256


DynamicState(pos: [0.86147153 0.16598971 0.6782485 ], orient: [0.76829654 0.49323863 0.4079435  0.00426822], lin_vel: [0. 0. 0.], ang_vel: [0. 0. 0.])
DynamicState(pos: [0.5057881  0.1629679  0.07500024], orient: [0.30302817 0.8370332  0.11889192 0.4397886 ], lin_vel: [0. 0. 0.], ang_vel: [0. 0. 0.])
DynamicState(pos: [0.872265   0.46508148 0.4845739 ], orient: [0.07897245 0.38311934 0.23072669 0.8909254 ], lin_vel: [0. 0. 0.], ang_vel: [0. 0. 0.])
DynamicState(pos: [0.28291336 0.73934186 0.8060379 ], orient: [0.40727636 0.63116837 0.64138734 0.15612468], lin_vel: [0. 0. 0.], ang_vel: [0. 0. 0.])
DynamicState(pos: [0.30428192 0.95970225 0.86471593], orient: [0.83802456 0.5084706  0.03519268 0.19476654], lin_vel: [0. 0. 0.], ang_vel: [0. 0. 0.])
DynamicState(pos: [0.7828553  0.86366415 0.3420988 ], orient: [0.51596355 0.01694572 0.5984859  0.6126248 ], lin_vel: [0. 0. 0.], ang_vel: [0. 0. 0.])
DynamicState(pos: [0.6941018  0.37077904 0.62912315], orient: [0.71009296 0.20135626 0.3305847

In [9]:
world.reset(soft=True)

# directly set usd xformOp attributes can update physics dynamic state
for _ in range(10):
    p = np.random.rand(7)
    cube.prim.GetAttribute('xformOp:translate').Set(Gf.Vec3d(p[0], p[1], p[2]))
    cube.prim.GetAttribute('xformOp:orient').Set(Gf.Quatd(p[3], p[4], p[5], p[6]))
    world.step()
    print(cube.get_current_dynamic_state())

2024-09-09 01:50:43 [470,708ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setLinearVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 232
2024-09-09 01:50:43 [470,708ms] [Error] [omni.physx.plugin] PhysX error: PxRigidDynamic::setAngularVelocity: Body must be non-kinematic!, FILE /builds/omniverse/physics/physx/source/physx/src/NpRigidDynamic.cpp, LINE 256


DynamicState(pos: [0.33879408 0.87230283 0.2695201 ], orient: [0.09626992 0.28048766 0.7620482  0.5756228 ], lin_vel: [-10.305921  40.2675    13.67939 ], ang_vel: [ -5.2982693 -25.957834   24.666277 ])
DynamicState(pos: [0.291449   0.8401678  0.08944554], orient: [0.3239697  0.3344021  0.7232938  0.50996566], lin_vel: [ -2.8407042  -1.9281005 -10.804474 ], ang_vel: [-10.474258 -15.492544 -22.953634])
DynamicState(pos: [0.37510988 0.03289961 0.82523894], orient: [0.85252106 0.28838578 0.28389484 0.330825  ], lin_vel: [  5.019652 -48.43609   44.147602], ang_vel: [-12.754311 -64.09503  -57.927273])
DynamicState(pos: [0.23087479 0.8058634  0.97916067], orient: [0.22350632 0.12107244 0.89937764 0.3556772 ], lin_vel: [-8.654105 46.377823  9.235303], ang_vel: [-21.963142  89.18368   63.22741 ])
DynamicState(pos: [0.9973416  0.91247004 0.32906473], orient: [0.61033976 0.21062125 0.6482628  0.4035832 ], lin_vel: [ 45.988007    6.3963995 -39.005753 ], ang_vel: [ 13.162733 -47.12305  -29.64905 ])